In [1]:
# KoBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [2]:
# Transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [4]:
# GPU 사용
device = torch.device("cuda:0") 

# CPU 사용 시
# device = torch.device("cpu") 

In [5]:
device

device(type='cuda', index=0)

In [6]:
# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /home/ai/ai/PAPER/.cache/kobert_v1.zip
using cached model. /home/ai/ai/PAPER/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
# [AI Hub] 감정 분류를 위한 대화 음성 데이터셋 불러오기
data = pd.read_csv("./total_df.csv")

# 2. 데이터 전처리

In [8]:
data.head()

,Unnamed: 0,chat,emotion
0,0,헐! 나 이벤트에 당첨 됐어.,happiness
1,1,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness
2,2,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness
3,3,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness
4,4,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral


In [9]:
total_df = data[['chat','emotion']]

In [10]:
total_df.head()

,chat,emotion
0,헐! 나 이벤트에 당첨 됐어.,happiness
1,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness
2,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness
3,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness
4,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral


In [11]:
total_df.shape

(207662, 2)

In [12]:
total_df['emotion'].unique()

array(['happiness', 'neutral', 'sadness', 'angry', 'surprise', 'disgust',
       'fear', '분노', '기쁨', '불안', '당황', '슬픔', '상처', '무감정'], dtype=object)

In [13]:
total_df.loc[total_df['emotion'] == '기쁨', 'emotion'] = 'happiness'

/tmp/ipykernel_24895/502879779.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.loc[total_df['emotion'] == '기쁨', 'emotion'] = 'happiness'


In [14]:
total_df.loc[total_df['emotion'] == '무감정', 'emotion'] = 'neutral'

/tmp/ipykernel_24895/3537383795.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.loc[total_df['emotion'] == '무감정', 'emotion'] = 'neutral'


In [15]:
total_df.loc[total_df['emotion'] == '슬픔', 'emotion'] = 'sadness'

/tmp/ipykernel_24895/1467515730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.loc[total_df['emotion'] == '슬픔', 'emotion'] = 'sadness'


In [16]:
total_df.loc[total_df['emotion'] == '분노', 'emotion'] = 'angry'

/tmp/ipykernel_24895/4215857237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.loc[total_df['emotion'] == '분노', 'emotion'] = 'angry'


In [17]:
total_df.loc[total_df['emotion'] == '당황', 'emotion'] = 'surprise'

/tmp/ipykernel_24895/1566104869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.loc[total_df['emotion'] == '당황', 'emotion'] = 'surprise'


In [18]:
total_df.loc[total_df['emotion'] == '불안', 'emotion'] = 'fear'

/tmp/ipykernel_24895/1431314182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.loc[total_df['emotion'] == '불안', 'emotion'] = 'fear'


In [19]:
total_df.loc[total_df['emotion'] == '상처', 'emotion'] = 'disgust'

/tmp/ipykernel_24895/189831862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.loc[total_df['emotion'] == '상처', 'emotion'] = 'disgust'


In [20]:
total_df['emotion'].unique()

array(['happiness', 'neutral', 'sadness', 'angry', 'surprise', 'disgust',
       'fear'], dtype=object)

In [21]:
total_df.to_csv('./total_df_convert.csv')

In [22]:
# 7개의 감정 class → 숫자
total_df.loc[(total_df['emotion'] == "fear"), 'emotion'] = 0  # fear → 0
total_df.loc[(total_df['emotion'] == "surprise"), 'emotion'] = 1  # surprise → 1
total_df.loc[(total_df['emotion'] == "angry"), 'emotion'] = 2  # angry → 2
total_df.loc[(total_df['emotion'] == "sadness"), 'emotion'] = 3  # sadness → 3
total_df.loc[(total_df['emotion'] == "neutral"), 'emotion'] = 4  # neutral → 4
total_df.loc[(total_df['emotion'] == "happiness"), 'emotion'] = 5  # happiness → 5
total_df.loc[(total_df['emotion'] == "disgust"), 'emotion'] = 6  # disgust → 6

/tmp/ipykernel_24895/3596059307.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.loc[(total_df['emotion'] == "fear"), 'emotion'] = 0  # fear → 0
/tmp/ipykernel_24895/3596059307.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.loc[(total_df['emotion'] == "surprise"), 'emotion'] = 1  # surprise → 1
/tmp/ipykernel_24895/3596059307.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df.loc[(total_df['emotion']

In [23]:
total_df['emotion'].unique()

array([5, 4, 3, 2, 1, 6, 0], dtype=object)

In [24]:
# [발화문, 상황] data_list 생성
data_list = []
for ques, label in zip (total_df['chat'], total_df['emotion']):
  data = []
  data.append(ques)
  data.append(str(label))

  data_list.append(data)

In [25]:
print(data)
print(data_list[:10])

['마찬가지야, 깨달음을 얻어 생사를 벗어난다 해도.', '4']
[['헐! 나 이벤트에 당첨 됐어.', '5'], ['내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.', '5'], ['한 명 뽑는 거였는데, 그게 바로 내가 된 거야.', '5'], ['당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. 그래서 못 산 향수야.', '5'], ['에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.', '4'], ['난 부페 형식의 음식들도 정말 좋아해. 그 것도 좀 알려 줘.', '4'], ['응. 완전히 끝난 거야. 한 달 동안 주말에 쉬지도 못하고 일만 했거든.', '5'], ['신나는 음악 듣는 것도 좋고, 어디 여행 가고 싶고 이 것 저 것 다 해보고 싶어.', '5'], ['친구들도 내 연락 기다리고 있을 텐데 내가 까먹고 있었네?', '5'], ['그래. 일단은 친구들부터 만나서 여행 계획에 대해서 얘기 좀 해봐야 되겠어.', '5']]


# 2-1.Split train & test data set

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)

In [28]:
print(len(dataset_train), len(dataset_test))

166129 41533


# 2-2.입력 데이터셋 토큰화

In [29]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

using cached model. /home/ai/ai/PAPER/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [30]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
# 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab = vocab, pad = pad, pair = pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

# 2-3.Setting parameters

In [31]:
# parameter 값 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

# 2-4.tokenization, int encoding, padding

In [32]:
# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [33]:
# torch 형식의 dataset을 만들어주면서, 입력 데이터셋의 전처리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

# 3.KoBERT 모델 구현

In [34]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,   # 클래스 수 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [35]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [36]:
# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

In [37]:
# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 4.Train

In [38]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/tmp/ipykernel_24895/17066059.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/2596 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9845285415649414 train acc 0.125
epoch 1 batch id 201 loss 1.926171898841858 train acc 0.16402363184079602
epoch 1 batch id 401 loss 1.7320576906204224 train acc 0.1925654613466334
epoch 1 batch id 601 loss 1.6617203950881958 train acc 0.21667013311148087
epoch 1 batch id 801 loss 1.552905559539795 train acc 0.24079275905118602
epoch 1 batch id 1001 loss 1.3918482065200806 train acc 0.26287774725274726
epoch 1 batch id 1201 loss 1.4500925540924072 train acc 0.2878200457951707
epoch 1 batch id 1401 loss 1.2333296537399292 train acc 0.3124219307637402
epoch 1 batch id 1601 loss 1.1572463512420654 train acc 0.336079013116802
epoch 1 batch id 1801 loss 1.2057034969329834 train acc 0.35904532204330925
epoch 1 batch id 2001 loss 1.0680456161499023 train acc 0.3812078335832084
epoch 1 batch id 2201 loss 1.0428285598754883 train acc 0.4028282598818719
epoch 1 batch id 2401 loss 0.8436154723167419 train acc 0.4235149416909621
epoch 1 train acc 0.4425999526351373


/tmp/ipykernel_24895/17066059.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/649 [00:00<?, ?it/s]

...

epoch 49 test acc 0.8803364728838818


  0%|          | 0/2596 [00:00<?, ?it/s]

epoch 50 batch id 1 loss 1.7689015294308774e-05 train acc 1.0
epoch 50 batch id 201 loss 1.926300501509104e-05 train acc 0.9989116915422885
epoch 50 batch id 401 loss 2.396802301518619e-05 train acc 0.9987920822942643
epoch 50 batch id 601 loss 1.570741915202234e-05 train acc 0.9989080698835274
epoch 50 batch id 801 loss 1.595698995515704e-05 train acc 0.999083177278402
epoch 50 batch id 1001 loss 0.0006871878285892308 train acc 0.9991258741258742
epoch 50 batch id 1201 loss 2.458513881720137e-05 train acc 0.99920639050791
epoch 50 batch id 1401 loss 1.8553035260993056e-05 train acc 0.9992193076374019
epoch 50 batch id 1601 loss 1.8247765183332376e-05 train acc 0.9991704403497814
epoch 50 batch id 1801 loss 2.0086275981157087e-05 train acc 0.9991844808439756
epoch 50 batch id 2001 loss 1.832777343224734e-05 train acc 0.9991957146426786
epoch 50 batch id 2201 loss 0.00597362220287323 train acc 0.9991907087687415
epoch 50 batch id 2401 loss 2.4723243768676184e-05 train acc 0.999186536859

  0%|          | 0/649 [00:00<?, ?it/s]

epoch 50 test acc 0.8803123973831115


In [41]:
PATH = './models/'
torch.save(model.state_dict(), PATH + 'kobert_new_nlp_epoch_50.pt')

# 5.Test

In [42]:
# predict : 학습 모델을 활용하여 다중 분류된 클래스를 출력해주는 함수
# 코드 출처 : https://hoit1302.tistory.com/159

def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환
    
    model.eval() 

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        # print('out = ', out)

        logit = out[0]
        softmax_logit = torch.softmax(logit, dim=-1)
        softmax_logit = softmax_logit.squeeze()

        max_index = torch.argmax(softmax_logit).item()
        max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()

        # print( 'max_index_value = ', max_index_value )
        
        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            # print('i = ', i)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + "(" + str(max_index_value) + ") 느껴집니다.")

In [43]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 :  안녕!


>> 입력하신 내용에서 행복이(0.9999961853027344) 느껴집니다.




하고싶은 말을 입력해주세요 :  앗 이거 너무 우낀데...


>> 입력하신 내용에서 행복이(0.999981164932251) 느껴집니다.




하고싶은 말을 입력해주세요 :  앗 ㅋㅋ 너무 웃겨


...

하고싶은 말을 입력해주세요 :  학교


>> 입력하신 내용에서 중립이(0.6105743050575256) 느껴집니다.




하고싶은 말을 입력해주세요 :  날이군


>> 입력하신 내용에서 슬픔이(0.9999918937683105) 느껴집니다.




KeyboardInterrupt: Interrupted by user